<a href="https://colab.research.google.com/github/Saxumatrum/Saxumatrum/blob/main/Titanic_Survivor_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
print('Service Libraries Imported')

Service Libraries Imported


In [35]:
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras import utils
from keras import regularizers
print('Model stuff imported')

Model stuff imported


In [36]:
df_train=pd.read_csv('train.csv')

In [37]:
df_train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [38]:
df_test=pd.read_csv('test.csv')

In [39]:
train_copy = df_train.copy()
test_copy = df_test.copy()

In [40]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [41]:
df_train=df_train.drop(['Name', 'Ticket','PassengerId','Embarked','Cabin','Fare'],axis=1)
df_test=df_test.drop(['Name', 'Ticket','PassengerId','Embarked','Cabin','Fare'],axis=1)

In [42]:
nan_count_per_column = df_train.isna().sum()

print(nan_count_per_column)

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64


In [43]:
#fill nan columns in age
average_age= df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(average_age)

In [44]:
nan_count_per_column = df_test.isna().sum()

print(nan_count_per_column)

Pclass     0
Sex        0
Age       86
SibSp      0
Parch      0
dtype: int64


In [45]:
#fill nan columns in age
average_age= df_test['Age'].mean()
df_test['Age'] = df_test['Age'].fillna(average_age)

In [46]:
#one-hot encode nominal values for train
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
# encodes and replaces the original values on the dataframe
values = enc.fit_transform(df_train.loc[:,['Sex']])
# adds the columns to the dataframe
df_train['F'] = values[:, 0]
df_train['M'] = values[:, 1]
# removes the categorical column
df_train = df_train.drop(columns=['Sex'])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [47]:
#one-hot encode nominal values for test
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
# encodes and replaces the original values on the dataframe
values = enc.fit_transform(df_test.loc[:,['Sex']])
# adds the columns to the dataframe
df_test['F'] = values[:, 0]
df_test['M'] = values[:, 1]
# removes the categorical column
df_test = df_test.drop(columns=['Sex'])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [48]:
df_train.head(5)

,Survived,Pclass,Age,SibSp,Parch,F,M
0,0,3,22.0,1,0,0.0,1.0
1,1,1,38.0,1,0,1.0,0.0
2,1,3,26.0,0,0,1.0,0.0
3,1,1,35.0,1,0,1.0,0.0
4,0,3,35.0,0,0,0.0,1.0


In [49]:
train_features = df_train.drop('Survived', axis=1)  # Drop the target column
train_labels = df_train['Survived']

# Convert features to a 2D NumPy array
train_features_array = train_features.values

# Reshape features to a 1D array
train_features_flat = train_features_array.reshape((train_features_array.shape[0], -1))

#Convert data types
train_features_flat = train_features_flat.astype('float32')
train_labels = train_labels.astype('float32')

In [50]:

# Create a fully connected neural network
model1 = Sequential()
model1.add(Dense(64, input_shape=(train_features_flat.shape[1],), activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

# Compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])




In [51]:
# Train the model
model1.fit(train_features_flat, train_labels, epochs=50, validation_split=0.2)

Epoch 1/50
23/23 [==============================] - 1s 11ms/step - loss: 0.6745 - accuracy: 0.6264 - val_loss: 0.6071 - val_accuracy: 0.6480
Epoch 2/50
23/23 [==============================] - 0s 5ms/step - loss: 0.6256 - accuracy: 0.6728 - val_loss: 0.5879 - val_accuracy: 0.6480
Epoch 3/50
23/23 [==============================] - 0s 5ms/step - loss: 0.6092 - accuracy: 0.6742 - val_loss: 0.5587 - val_accuracy: 0.6536
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5832 - accuracy: 0.7107 - val_loss: 0.5237 - val_accuracy: 0.6704
Epoch 5/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5790 - accuracy: 0.7093 - val_loss: 0.4920 - val_accuracy: 0.8045
Epoch 6/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5449 - accuracy: 0.7219 - val_loss: 0.4720 - val_accuracy: 0.8156
Epoch 7/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5222 - accuracy: 0.7767 - val_loss: 0.4828 - val_accuracy: 0.7263
Epoch 8/50
23/23 [=

In [57]:
#PREDICTIONS
test_features = df_test

# Convert test features to a 2D NumPy array
test_features_array = test_features.values

# Reshape test features to a 1D array
test_features_flat = test_features_array.reshape((test_features_array.shape[0], -1))

# Convert data types
test_features_flat = test_features_flat.astype('float32')

# Make predictions on the test set
test_predictions = model1.predict(test_features_flat)

#using 0.5 as threshold because of activation function
binary_predictions = (test_predictions > 0.5).astype(int)

# Create a DataFrame with test predictions
result_df = pd.DataFrame({'Survived': binary_predictions.flatten()})

# Concatenate the test features and predictions (optional)
result_df = pd.concat([test_copy['PassengerId'], result_df], axis=1)

# Save the DataFrame to a CSV file
result_df.to_csv('titanic_predictions.csv', index=False)

14/14 [==============================] - 0s 3ms/step


In [58]:
df=pd.read_csv('titanic_predictions.csv')

In [59]:
df.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
